# Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "llama3.1","api_type":"ollama"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task ,"role": "user"}])

In [5]:
print(reply)

{'content': "**Unlocking the Power of Hope: The Revolutionary AI Assistant**\n\nHope AI is changing the game for individuals and businesses alike with its cutting-edge technology. This intuitive assistant uses natural language processing to provide personalized solutions, automating mundane tasks and freeing up time for more important pursuits. By leveraging machine learning algorithms, Hope AI learns your preferences and adapts its services accordingly. Whether you're looking to streamline workflows or boost productivity, this innovative tool is the perfect companion. Say goodbye to tedious tasks and hello to a more efficient, stress-free future with Hope AI by your side.", 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking Endless Possibilities: The Power of Hope AI**

Imagine a world where artificial intelligence not only assists, but also empowers. Welcome to the realm of Hope AI, where machines learn from human emotions and create innovative solutions for a better tomorrow. With its cutting-edge technology, Hope AI is revolutionizing industries, improving lives, and giving new meaning to the phrase "intelligent assistance". This forward-thinking platform combines human ingenuity with AI's processing power to unlock endless possibilities. As we step into this exciting future, one thing is clear: hope has a new name – Hope AI.

--------------------------------------------------------------------------------
Critic (to Writer):

**Constructive F

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking Endless Possibilities: The Power of Hope AI**

Imagine a world where artificial intelligence not only assists, but also empowers. Welcome to the realm of Hope AI, where machines learn from human emotions and create innovative solutions for a better tomorrow. With its cutting-edge technology, Hope AI is revolutionizing industries, improving lives, and giving new meaning to the phrase "intelligent assistance". This forward-thinking platform combines human ingenuity with AI's processing power to unlock endless possibilities. As we step into this exciting future, one thing is clear: hope has a new name – Hope AI.

--------------------------------------------------------------------------------

************************************

## Get the summary

In [15]:
print(res.summary)

**Unlocking Endless Possibilities with Hope AI**

Imagine a world where artificial intelligence not only assists, but also empowers. Welcome to the realm of Hope AI, where machines learn from human emotions and create innovative solutions for a better tomorrow. With its cutting-edge technology, Hope AI is revolutionizing industries, improving lives, and giving new meaning to the phrase "intelligent assistance". We want to assure our users that Hope AI prioritizes user data protection, adhering to strict policies on collection, storage, and usage. As we step into this exciting future, one thing is clear: hope has a new name – Hope AI.
